In [1]:
import os
import glob
import cudf
import dask_cudf
import hydra

import sys
sys.path.append('../../..')
from aiagents4pharma.talk2knowledgegraphs.utils.embeddings.ollama import EmbeddingWithOllama

/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/awmulya

In [2]:
# Load hydra configuration
with hydra.initialize(version_base=None, config_path="../../../aiagents4pharma/talk2knowledgegraphs/configs"):
    cfg = hydra.compose(
        config_name="config", overrides=["tools/multimodal_subgraph_extraction=default"]
    )
    cfg = cfg.tools.multimodal_subgraph_extraction
cfg

{'_target_': 'talk2knowledgegraphs.tools.multimodal_subgraph_extraction', 'ollama_embeddings': ['nomic-embed-text'], 'temperature': 0.1, 'streaming': False, 'topk': 5, 'topk_e': 5, 'cost_e': 0.5, 'c_const': 0.01, 'root': -1, 'num_clusters': 1, 'pruning': 'gw', 'verbosity_level': 0, 'node_id_column': 'node_id', 'node_attr_column': 'node_attr', 'edge_src_column': 'edge_src', 'edge_attr_column': 'edge_attr', 'edge_dst_column': 'edge_dst', 'node_colors_dict': {'gene/protein': '#6a79f7', 'molecular_function': '#82cafc', 'cellular_component': '#3f9b0b', 'biological_process': '#c5c9c7', 'drug': '#c4a661', 'disease': '#80013f'}, 'biobridge': {'source': 'aiagents4pharma/talk2knowledgegraphs/tests/files/ibd_biobridge_multimodal/', 'node_type': ['gene/protein', 'molecular_function', 'cellular_component', 'biological_process', 'drug', 'disease']}}

In [3]:
cfg.biobridge.source = "aiagents4pharma/talk2knowledgegraphs/tests/files/ibd_biobridge_multimodal"

In [4]:
# Loop over nodes and edges
graph_dict = {}
for element in ["nodes", "edges"]:
    # Make an empty dictionary for each folder
    graph_dict[element] = {}
    for stage in ["enrichment", "embedding"]:
        print(element, stage)
        # Create the file pattern for the current subfolder
        file_list = glob.glob(os.path.join('../../../', 
                                           cfg.biobridge.source, 
                                           element, 
                                           stage, '*.parquet.gzip'))
        # if element != "edges" and stage == "embedding":
        # Read and concatenate all dataframes in the folder
        graph_dict[element][stage] = cudf.concat([cudf.read_parquet(f) for f in file_list], ignore_index=True)

nodes enrichment
nodes embedding
edges enrichment
edges embedding


In [5]:
# unified_nodes_df = graph_dict["nodes"]["enrichment"].merge(
#     graph_dict["nodes"]["embedding"],
#     on=["node_id"],
#     how="left"
# )
# unified_nodes_df.head(5)

In [6]:
# # Perform mini batch merging of edges 
# from tqdm import tqdm
# import pandas as pd

# mini_batch_size = 10000
# df = pd.DataFrame()
# # Iterate over the edges in mini-batches
# for i in tqdm(range(0, graph_dict["edges"]["enrichment"].shape[0], mini_batch_size)):
#     u_df = graph_dict["edges"]["enrichment"][i:i+mini_batch_size].merge(
#         graph_dict["edges"]["embedding"],
#         on=["edge_type_str"],
#         how="left"
#     ).to_pandas()
#     df = pd.concat([df, u_df], ignore_index=True)
# df.head(5)

In [7]:
# from tqdm import tqdm
# import pandas as pd

# mini_batch_size = 10000
# dfs = []

# for i in tqdm(range(0, graph_dict["edges"]["enrichment"].shape[0], mini_batch_size)):
#     batch_df = graph_dict["edges"]["enrichment"][i:i+mini_batch_size].merge(
#         graph_dict["edges"]["embedding"],
#         on="edge_type_str",
#         how="left"
#     ).to_pandas()
#     dfs.append(batch_df)

# # One-time concat (much faster)
# unified_edges_df = pd.concat(dfs, ignore_index=True)
# unified_edges_df

In [8]:
# unified_edges_df = graph_dict["edges"]["enrichment"].merge(
#     graph_dict["edges"]["embedding"],
#     on=["edge_type_str"],
#     how="left"
# )

In [9]:
# import torch
# from torch_geometric.data import TensorAttr
# from cugraph_pyg.data import CuGraphStore, TensorDictFeatureStore

# # Create and initialize FeatureStore for each column that can be used as a feature
# mapper = {}
# fs = TensorDictFeatureStore()
# for nt in graph_dict['nodes']['enrichment'].node_type.unique().to_arrow().to_pylist():
#     print(f"Node type: {nt}")
#     nodes_df = graph_dict['nodes']['enrichment'][graph_dict['nodes']['enrichment'].node_type == nt]
#     print(f"Number of nodes: {len(nodes_df)}")

#     # Node Index
#     node_index = nodes_df["node_index"].to_arrow().to_pylist()
#     mapper[nt] = {}
#     mapper[nt]['to_node_index'] = {i: node_index[i] for i in range(len(node_index))}
#     mapper[nt]['from_node_index'] = {node_index[i]: i for i in range(len(node_index))}
#     fs[TensorAttr(group_name=nt, 
#                     attr_name="node_index")] = torch.tensor(node_index, dtype=torch.int64)
    
#     # Node Features
#     nodes_emb_df = graph_dict['nodes']['embedding'][graph_dict['nodes']['embedding'].node_id.isin(nodes_df.node_id)]
#     fs[TensorAttr(group_name=nt,
#                   attr_name="desc_emb")] = torch.tensor(nodes_emb_df["desc_emb"].to_arrow().to_pylist(), dtype=torch.float32)
#     fs[TensorAttr(group_name=nt,
#                   attr_name="feat_emb")] = torch.tensor(nodes_emb_df["feat_emb"].to_arrow().to_pylist(), dtype=torch.float32)


In [10]:
# # Get nodes enrichment and embedding dataframes
# enrichment_df = graph_dict['nodes']['enrichment']
# embedding_df = graph_dict['nodes']['embedding']

# # Convert node_id columns once to set
# embedding_node_ids = set(embedding_df['node_id'].to_pandas())


In [11]:
# import torch
# import cudf
# from torch_geometric.data import TensorAttr
# from cugraph_pyg.data import TensorDictFeatureStore

# # Initialize FeatureStore and mapper
# fs = TensorDictFeatureStore()
# mapper = {}

# # Get nodes enrichment and embedding dataframes
# nodes_enrichment_df = graph_dict['nodes']['enrichment']
# nodes_embedding_df = graph_dict['nodes']['embedding']

# # Get node_ids
# embedding_node_ids = set(nodes_embedding_df['node_id'].to_pandas())

# # Loop over group enrichment nodes by type 
# for nt, nodes_df in nodes_enrichment_df.groupby('node_type'):
#     print(f"Node type: {nt}")
#     node_count = len(nodes_df)
#     print(f"Number of nodes: {node_count}")

#     # Get node_index as torch tensor directly
#     node_index_tensor = torch.tensor(nodes_df["node_index"].to_numpy(), dtype=torch.int64)
#     fs[TensorAttr(group_name=nt, attr_name="node_index")] = node_index_tensor

#     # Construct mapper for node_index
#     node_index_list = node_index_tensor.tolist()
#     mapper[nt] = {
#         'to_node_index': dict(enumerate(node_index_list)),
#         'from_node_index': {v: i for i, v in enumerate(node_index_list)}
#     }

#     # Filter embeddings once by node_id
#     mask = nodes_embedding_df['node_id'].isin(set(nodes_df['node_id'].to_pandas()))
#     nodes_emb_df = nodes_embedding_df[mask]

#     # Convert embeddings as tensors and add to FeatureStore
#     for attr_name in ["desc_emb", "feat_emb"]:
#         emb_tensor = torch.tensor(nodes_emb_df[attr_name].to_arrow().to_pylist(), dtype=torch.float32)
#         fs[TensorAttr(group_name=nt, attr_name=attr_name)] = emb_tensor


In [12]:
import torch
import cudf
from torch_geometric.data import TensorAttr
from cugraph_pyg.data import GraphStore, TensorDictFeatureStore
# cugraph_pyg.data.GraphStore()
# Initialize FeatureStore and mapper
fs = TensorDictFeatureStore()
mapper = {}

# Get nodes enrichment and embedding dataframes
nodes_enrichment_df = graph_dict['nodes']['enrichment']
nodes_embedding_df = graph_dict['nodes']['embedding']

# Loop over group enrichment nodes by type 
for nt, nodes_df in nodes_enrichment_df.groupby('node_type'):
    print(f"Node type: {nt}")
    node_count = len(nodes_df)
    print(f"Number of nodes: {node_count}")


    # Get node_ids
    emb_df = nodes_embedding_df[nodes_embedding_df['node_id'].isin(nodes_df['node_id'])]

    # Sort both by node_id for alignment
    nodes_df_sorted = nodes_df.sort_values('node_id')
    emb_df_sorted = emb_df.sort_values('node_id')

    # Ensure sorted node_ids match
    assert cudf.Series.equals(nodes_df_sorted['node_id'].reset_index(drop=True),
                              emb_df_sorted['node_id'].reset_index(drop=True)), \
                                f"Node ID mismatch in {nt} after sorting"

    # Get node_index as torch tensor directly
    node_index_tensor = torch.tensor(nodes_df_sorted["node_index"].to_numpy(), dtype=torch.int64)
    fs[TensorAttr(group_name=nt, attr_name="node_index")] = node_index_tensor

    # # Construct mapper for node_index
    node_index_list = nodes_df_sorted["node_index"].to_numpy().tolist()
    mapper[nt] = {
        'to_node_index': dict(enumerate(node_index_list)),
        'from_node_index': {v: i for i, v in enumerate(node_index_list)}
    }

    # Convert embeddings as tensors and add to FeatureStore
    for attr_name in ["desc_emb", "feat_emb"]:
        emb_tensor = torch.tensor(emb_df_sorted[attr_name].to_arrow().to_pylist(), dtype=torch.float32)
        fs[TensorAttr(group_name=nt, attr_name=attr_name)] = emb_tensor


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/__init__.py:27: FutureWarning: TensorDictFeatureStore is deprecated.  Consider changing your workflow to launch using 'torchrun' and store data in the faster and more memory-efficient WholeFeatureStore instead.
  warnings.warn(
INFO:numba.cuda.cudadrv.driver:init


Node type: biological_process
Number of nodes: 1615


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group biological_process)
  warnings.warn(


Node type: cellular_component
Number of nodes: 202
Node type: disease
Number of nodes: 7
Node type: drug
Number of nodes: 748


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group cellular_component)
  warnings.warn(
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group disease)
  warnings.warn(
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group drug)
  warnings.warn(


Node type: gene/protein
Number of nodes: 102
Node type: molecular_function
Number of nodes: 317


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group gene/protein)
  warnings.warn(
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group molecular_function)
  warnings.warn(


In [13]:
# Initialize GraphStore
gs = GraphStore()

# Get edges enrichment and embedding dataframes
edges_enrichment_df = graph_dict['edges']['enrichment']
edges_embedding_df = graph_dict['edges']['embedding']

# Loop over edge types 
for edge_type_str in edges_enrichment_df['edge_type_str'].unique().to_arrow().to_pylist():
    print(f"Processing edge type: {edge_type_str}")
    src_type, rel_type, tgt_type = edge_type_str.split('|')

    # Filter edges for this edge_type_str once
    filtered_df = edges_enrichment_df[edges_enrichment_df['edge_type_str'] == edge_type_str][['triplet_index', 'head_index', 'tail_index']]

    # Convert mapper dicts to cudf Series for vectorized mapping
    src_map = cudf.Series(mapper[src_type]['from_node_index'])
    tgt_map = cudf.Series(mapper[tgt_type]['from_node_index'])

    # Vectorized mapping of head_index and tail_index using replace (works like dict lookup)
    mapped_head = filtered_df['head_index'].replace(src_map).astype('int64')
    mapped_tail = filtered_df['tail_index'].replace(tgt_map).astype('int64')

    # Check if mapping was successful
    if mapped_head.isnull().any() or mapped_tail.isnull().any():
            raise ValueError(f"Mapping failure for edge type {edge_type_str}")

    # Edge index
    edge_index = torch.tensor(
        cudf.concat([mapped_head, mapped_tail], axis=1).to_pandas().values.T,
        dtype=torch.long
    ).contiguous()

    # Store edge index in the GraphStore
    gs[(src_type, rel_type, tgt_type), "coo"] = edge_index

    # Add triplet index to the FeatureStore
    triplet_index_tensor = torch.tensor(filtered_df['triplet_index'].to_numpy(), dtype=torch.long).unsqueeze(0)
    fs[TensorAttr(group_name=(src_type, rel_type, tgt_type), attr_name='triplet_index')] = triplet_index_tensor

    # Store edge embeddings in the FeatureStore
    edge_emb_df = edges_embedding_df[edges_embedding_df['edge_type_str'] == edge_type_str]

    # Convert edge embeddings to torch tensor
    edge_emb_tensor = torch.tensor(edge_emb_df['edge_emb'].to_arrow().to_pylist(), dtype=torch.float32).unsqueeze(0)
    fs[TensorAttr(group_name=(src_type, rel_type, tgt_type), attr_name='edge_emb')] = edge_emb_tensor



Processing edge type: drug|carrier|gene/protein
Processing edge type: drug|enzyme|gene/protein
Processing edge type: drug|target|gene/protein
Processing edge type: drug|transporter|gene/protein


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('drug', 'carrier', 'gene/protein'))
  warnings.warn(
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('drug', 'enzyme', 'gene/protein'))
  warnings.warn(
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('drug', 'target', 'gene/protein'))
  warnings.warn(
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('drug', 'transporter', 

Processing edge type: gene/protein|associated with|disease
Processing edge type: gene/protein|interacts with|molecular_function


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('gene/protein', 'associated with', 'disease'))
  warnings.warn(
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('gene/protein', 'interacts with', 'molecular_function'))
  warnings.warn(


Processing edge type: gene/protein|interacts with|cellular_component
Processing edge type: gene/protein|interacts with|biological_process


...

/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('disease', 'associated with', 'gene/protein'))
  warnings.warn(
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('molecular_function', 'interacts with', 'gene/protein'))
  warnings.warn(


Processing edge type: molecular_function|interacts with|gene/protein
Processing edge type: cellular_component|interacts with|gene/protein


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('cellular_component', 'interacts with', 'gene/protein'))
  warnings.warn(


Processing edge type: biological_process|interacts with|gene/protein


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/feature_store.py:67: UserWarning: Ignoring index parameter (attribute does not exist for group ('biological_process', 'interacts with', 'gene/protein'))
  warnings.warn(


## Prepare Query DF


In [14]:
# Define the data path
DATA_PATH = "../../../aiagents4pharma/talk2knowledgegraphs/tests/files"

# Define the agent state
state = {
    "selections": {
        "gene/protein": [],
        "molecular_function": [],
        "cellular_component": [],
        "biological_process": [],
        "drug": [],
        "disease": []
    },
    "uploaded_files": [],
    "topk_nodes": 3,
    "topk_edges": 3,
    "dic_source_graph": [
        {
            "name": "BioBridge",
            "kg_pyg_path": f"{DATA_PATH}/biobridge_multimodal_pyg_graph.pkl",
            "kg_text_path": f"{DATA_PATH}/biobridge_multimodal_text_graph.pkl",
        }
    ],
}

In [15]:
# Update state
state["uploaded_files"] = [
    {
        "file_name": "multimodal-analysis.csv",
        "file_path": f"{DATA_PATH}/multimodal-analysis.csv",
        "file_type": "multimodal",
        "uploaded_by": "VPEUser",
        "uploaded_timestamp": "2025-05-12 00:00:00",
    }
]

In [16]:
# Parameters
prompt = """
Extract all relevant information related to nodes of genes related to inflammatory bowel disease
(IBD) that existed in the knowledge graph.
Please set the extraction name for this process as `subkg_12345`.
"""
tool_call_id = "subgraph_extraction_tool"
extraction_name = "subgraph_12345"

In [17]:

# Initialize dataframes
# logger.log(logging.INFO, "Initializing dataframes")
multimodal_df = cudf.DataFrame({"name": [], "node_type": []})
query_df = cudf.DataFrame({"node_id": [],
                            "node_type": [],
                            "x": [],
                            "desc_x": [],
                            "use_description": []})

# Loop over the uploaded files and find multimodal files
# logger.log(logging.INFO, "Looping over uploaded files")
for i in range(len(state["uploaded_files"])):
    # Check if multimodal file is uploaded
    if state["uploaded_files"][i]["file_type"] == "multimodal":
        # Read the csv file
        multimodal_df = cudf.read_csv(state["uploaded_files"][i]["file_path"])

In [20]:
graph_dict["nodes"]["enrichment"]

,node_index,primekg_node_index,node_id,node_name,node_type,desc,feat
0,1851,53517,methyltransferase activity_(53517),methyltransferase activity,molecular_function,methyltransferase activity belongs to molecula...,Catalysis of the transfer of a methyl group to...
1,1852,53528,protein serine/threonine kinase activity_(53528),protein serine/threonine kinase activity,molecular_function,protein serine/threonine kinase activity belon...,Catalysis of the reactions: ATP + protein seri...
2,1853,53551,carboxylic acid transmembrane transporter acti...,carboxylic acid transmembrane transporter acti...,molecular_function,carboxylic acid transmembrane transporter acti...,Enables the transfer of carboxylic acids from ...
3,1854,53616,efflux transmembrane transporter activity_(53616),efflux transmembrane transporter activity,molecular_function,efflux transmembrane transporter activity belo...,Enables the transfer of a specific substance o...
4,1855,53620,metal ion binding_(53620),metal ion binding,molecular_function,metal ion binding belongs to molecular_functio...,Binding to a metal ion.
...,...,...,...,...,...,...,...
2986,848,34781,IL17REL_(34781),IL17REL,gene/protein,IL17REL belongs to gene/protein node. IL17REL ...,MSRSVLEALTSSTAMQCVPSDGCAMLLRVRASITLHERLRGLEACA...
2987,849,34814,TAGAP_(34814),TAGAP,gene/protein,TAGAP belongs to gene/protein node. TAGAP is T...,MKLRSSHNASKTLNANNMETLIECQSEGDIKEHPLLASCESEDSIC...
2988,850,34887,DENND1B_(34887),DENND1B,gene/protein,DENND1B belongs to gene/protein node. DENND1B ...,MDCRTKANPDRTFDLVLKVKCHASENEDPVVLWKFPEDFGDQEILQ...
2989,851,34967,IL21_(34967),IL21,gene/protein,IL21 belongs to gene/protein node. IL21 is int...,MRSSPGNMERIVICLMVIFLGTLVHKSSSQGQDRHMIRMRQLIDIV...


In [32]:
edges_df_sorted = graph_dict["edges"]["enrichment"].sort_values("triplet_index", ignore_index=True)

In [35]:
import cupy as cp
cp.stack([
    edges_df_sorted["head_index"].to_cupy(),
    edges_df_sorted["tail_index"].to_cupy()
])

array([[ 123,   99,  320, ..., 2509, 2304, 2142],
       [  36,   47,   47, ...,   22,  838,  846]])

In [34]:
import cupy as cp
cp.stack([
    graph_dict["edges"]["enrichment"]["head_index"].to_cupy(),
    graph_dict["edges"]["enrichment"]["tail_index"].to_cupy()
])

array([[ 123,   99,  320, ..., 2509, 2304, 2142],
       [  36,   47,   47, ...,   22,  838,  846]])

In [ ]:
triplet_index_sorted = graph_dict['edges']['enrichment']['triplet_index'].sort_values(ignore_index=True)

In [25]:
prompt_emb = [EmbeddingWithOllama(model_name=cfg.ollama_embeddings[0]).embed_query(prompt)]
graph = graph_dict

# Check if the multimodal_df is empty
# logger.log(logging.INFO, "Checking if multimodal_df is empty")
if len(multimodal_df) > 0:
    # Prepare multimodal_df
    # logger.log(logging.INFO, "Preparing multimodal_df")
    multimodal_df.rename(columns={"name": "q_node_name",
                                    "node_type": "q_node_type"}, inplace=True)

    # Make and process a query dataframe by merging the graph_df and multimodal_df
    # logger.log(logging.INFO, "Processing query dataframe")
    query_df = graph["nodes"]["enrichment"][
        ['node_id', 'node_name', 'node_type', 'desc', 'feat']
    ].merge(multimodal_df, how='cross')
    # logger.log(logging.INFO, "Lowering case for node names (q_node_name)")
    query_df['q_node_name'] = query_df['q_node_name'].str.lower()
    # logger.log(logging.INFO, "Lowering case for node names (node_name)")
    query_df['node_name'] = query_df['node_name'].str.lower()
    # Get the mask for filtering based on the query
    # logger.log(logging.INFO, "Filtering based on the query")
    mask = (
        query_df['node_name'].str.contains(query_df['q_node_name']) &
        (query_df['node_type'] == query_df['q_node_type'])
    )
    query_df = query_df[mask]
    query_df = query_df[['node_id',
                         'node_type', 
                         'feat', 
                         'desc']].reset_index(drop=True)
    query_df['use_description'] = False # set to False for modal-specific embeddings

    # Merge the query dataframe with the node embeddings
    query_df = query_df.merge(
        graph["nodes"]["embedding"][["node_id", "desc_emb", "feat_emb"]],
        on="node_id",
        how="left"
    )[['node_id', 'node_type', 'feat', 'desc', 'desc_emb', 'feat_emb', 'use_description']]

    # Update the state by adding the the selected node IDs
    # logger.log(logging.INFO, "Updating state with selected node IDs")
    state["selections"] = query_df.to_pandas().groupby(
        "node_type"
    )["node_id"].apply(list).to_dict()

# Append a user prompt to the query dataframe
# logger.log(logging.INFO, "Adding user prompt to query dataframe")
query_df = cudf.concat([
    query_df,
    cudf.DataFrame({
        'node_id': 'user_prompt',
        'node_type': 'prompt',
        'feat': prompt,
        'desc': prompt,
        'feat_emb': prompt_emb,
        'desc_emb': prompt_emb,
        'use_description': True # set to True for user prompt embedding
    })
]).reset_index(drop=True)

INFO:httpx:HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


In [62]:
node_colors = {n: cfg.node_colors_dict[k]
                for k, v in state["selections"].items() for n in v}
color_df = cudf.DataFrame(list(node_colors.items()), columns=["node_id", "color"])
# color_df = cudf.DataFrame([], columns=["node_id", "color"])
color_df
 

,node_id,color
0,IL7R_(625),#6a79f7
1,TCF7_(5195),#6a79f7


In [64]:
graph_nodes = graph["nodes"]["enrichment"].sort_values("node_index", ignore_index=True).copy()

graph_nodes = graph_nodes.iloc[[5, 54]][
    ['node_id', 'node_name', 'node_type', 'desc', 'feat']
]
if not color_df.empty:
    # Merge the color dataframe with the graph nodes
    graph_nodes = graph_nodes.merge(color_df, on="node_id", how="left")
else:
    graph_nodes["color"] = 'black'  # Default color
graph_nodes['color'].fillna('black', inplace=True)
graph_nodes

,node_id,node_name,node_type,desc,feat,color
0,IL7R_(625),IL7R,gene/protein,IL7R belongs to gene/protein node. IL7R is int...,MTILGTTFGMVFSLLQVVSGESGYAQNGDLEDAELDDYSFSCYSQL...,#6a79f7
1,TCF7_(5195),TCF7,gene/protein,TCF7 belongs to gene/protein node. TCF7 is tra...,MPQLDSGGGGAGGGDDLGAPDELLAFQDEGEEQDDKSRDSAAGPER...,#6a79f7


In [26]:
query_df

,node_id,node_type,feat,desc,desc_emb,feat_emb,use_description
0,IL7R_(625),gene/protein,MTILGTTFGMVFSLLQVVSGESGYAQNGDLEDAELDDYSFSCYSQL...,IL7R belongs to gene/protein node. IL7R is int...,"[0.04506902, 0.008911126, -0.17318207, -0.0157...","[0.06364653259515762, 0.06951971352100372, 0.0...",False
1,TCF7_(5195),gene/protein,MPQLDSGGGGAGGGDDLGAPDELLAFQDEGEEQDDKSRDSAAGPER...,TCF7 belongs to gene/protein node. TCF7 is tra...,"[0.036997586, 0.038098544, -0.19027671, -0.006...","[0.028321774676442146, 0.003539376426488161, 0...",False
2,user_prompt,prompt,\nExtract all relevant information related to ...,\nExtract all relevant information related to ...,"[0.047626022, 0.05007147, -0.16455309, -0.0442...","[0.047626022, 0.05007147, -0.16455309, -0.0442...",True


## Retrieval 

In [27]:
# Nodes retrieval
prompt = """
Extract all relevant information related to nodes of genes related to inflammatory bowel disease (IBD) that existed in the knowledge graph.
In particular, extract a set of nodes and edges interconnecting the IL6, IL21, and TNF in relation to drugs called Mesalazine and Betamethasone for potentially treating the disease.
Please set the extraction name for this process as `subkg_12345`.

Tell me about interconnections that involve RELA node in the extracted graph.
"""

In [28]:
# Using nomic-ai/nomic-embed-text-v1.5 model via Ollama
emb_model = EmbeddingWithOllama(model_name='nomic-embed-text')

# Get embeddings for the prompt
prompt_emb = emb_model.embed_query(prompt)

INFO:httpx:HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"


In [29]:
import cupy as cp
from cuvs.distance import pairwise_distance

def _compute_sim_scores(features_a: cp.ndarray,
                        features_b:  cp.ndarray,
                        metric: str="cosine"):
        """
        Compute the similarity scores between two sets of features using the specified metric.

        Args:
            features_a: The first set of features.
            features_b: The second set of features.
            metric: The metric to use for computing the similarity scores.
        
        Returns:
            The similarity scores between the two sets of features.
        """
        scores = pairwise_distance(features_a, features_b, metric=metric)
        scores = 1 - cp.asarray(scores).ravel()
        return scores

In [ ]:
# query_emb = prompt_emb

# # All nodes similarity search
# sim_score_df = cudf.DataFrame({
#     'node_index': nodes_enrichment_df['node_index'],
#     'score' : cudf.Series(cp.zeros(nodes_enrichment_df.shape[0], dtype=cp.float32))
# }).sort_values('node_index').reset_index(drop=True)

# # Calculate cosine similarity for text features and update the score
# for nt in nodes_enrichment_df.node_type.unique().to_arrow().to_pylist():
#     node_emb = fs[TensorAttr(group_name=nt, attr_name='desc_emb', index=None)]
#     node_index = fs[TensorAttr(group_name=nt, attr_name='node_index', index=None)]
#     modal_score = _compute_sim_scores(
#         features_a=cp.array(node_emb).astype(cp.float32),
#         features_b=cp.array(query_emb).reshape(1, -1).astype(cp.float32),
#         metric="cosine"
#     )
#     modal_score_df = cudf.DataFrame({
#         'node_index': node_index,
#         'score': modal_score
#     }).sort_values('node_index').reset_index(drop=True)

#     # Update the sim_score_df with the new scores
#     sim_score_df = sim_score_df.merge(modal_score_df, on='node_index', how='left', suffixes=('', '_new'))
#     sim_score_df['score'] = sim_score_df['score_new'].fillna(sim_score_df['score'])
#     sim_score_df = sim_score_df.drop(columns=['score_new'])
#     sim_score_df = sim_score_df.sort_values('node_index').reset_index(drop=True)

# sim_score_df


,node_index,score
0,0,0.643276
1,1,0.675348
2,2,0.607963
3,3,0.611600
4,4,0.600781
...,...,...
2986,2986,0.517834
2987,2987,0.555667
2988,2988,0.537938
2989,2989,0.552041


### Nodes Retrieval

In [36]:
# Parameter:
use_description = False
modality = 'gene/protein'

query_emb = fs[TensorAttr(group_name=modality, attr_name='feat_emb', index=None)][0]
if use_description:
    query_emb = prompt_emb
    modality = 'prompt'

# ==============

# Modal-specific nodes similarity search
node_index_sorted = nodes_enrichment_df['node_index'].sort_values(ignore_index=True)
sim_score_df = cudf.DataFrame({
    'node_index': node_index_sorted,
    'score': cp.zeros(len(node_index_sorted), dtype=cp.float32)
})

# Determine modalities and feature to use
feat = 'desc_emb' if use_description else 'feat_emb'
modalities = nodes_enrichment_df['node_type'].unique().to_arrow().to_pylist() if use_description else [modality]

# Calculate cosine similarity for text features and update the score
score_map = {}
for nt in modalities:
    # print(f"Processing node type: {nt}")
    
    # Get node embeddings and indices
    node_emb = fs[TensorAttr(group_name=nt, attr_name=feat, index=None)]
    node_index = fs[TensorAttr(group_name=nt, attr_name='node_index', index=None)]

    # Perform similarity computation
    modal_score = _compute_sim_scores(
        features_a=cp.asarray(node_emb, dtype=cp.float32),
        features_b=cp.asarray(query_emb, dtype=cp.float32).reshape(1, -1),
        metric="cosine"
    )

    # Use CuPy to update scores in a flat buffer first
    score_map[nt] = cudf.DataFrame({
        'node_index': node_index,
        'score': modal_score
    })

# Combine all modal score dataframes in one go (faster than sequential merging)
if score_map:
    all_modal_scores = cudf.concat(score_map.values())
    sim_score_df = sim_score_df.merge(all_modal_scores, on='node_index', how='left', suffixes=('', '_new'))
    sim_score_df['score'] = sim_score_df['score_new'].fillna(sim_score_df['score'])
    sim_score_df = sim_score_df.drop(columns=['score_new'])
    sim_score_df = sim_score_df.sort_values('node_index').reset_index(drop=True)

sim_score_df


,node_index,score
0,0,0.962305
1,1,0.880833
2,2,0.927842
3,3,0.964579
4,4,0.953058
...,...,...
2986,2986,0.000000
2987,2987,0.000000
2988,2988,0.000000
2989,2989,0.000000


In [37]:
topk = 5

# Set the prizes for nodes based on the similarity scores
topk = min(topk, sim_score_df.shape[0])
n_prizes = cudf.Series(0.0, index=cp.arange(sim_score_df.shape[0]))
n_prizes[(-sim_score_df.score).sort_values()[:topk].index] = cp.arange(topk,
                                                                0, -1).astype(cp.float32)
n_prizes.unique()

0    0.0
1    1.0
2    5.0
3    4.0
4    3.0
5    2.0
dtype: float64

### Edges Retrieval

In [45]:
query_emb = prompt_emb

# Edges retrieval
triplet_index_sorted = edges_enrichment_df['triplet_index'].sort_values(ignore_index=True)
sim_score_df = cudf.DataFrame({
    'triplet_index': triplet_index_sorted,
    'score': cp.zeros(len(triplet_index_sorted), dtype=cp.float32)
})

# # Determine modalities and feature to use
# feat = 'desc_emb' if use_description else 'feat_emb'
# modalities = nodes_enrichment_df['node_type'].unique().to_arrow().to_pylist() if use_description else [modality]

# Calculate cosine similarity for text features and update the score
score_map = {}
for et in edges_enrichment_df.edge_type.unique().to_arrow().to_pylist():
    print(f"Processing edge type: {et}")
    
    # Get node embeddings and indices
    edge_emb = fs[TensorAttr(group_name=tuple(et), attr_name='edge_emb', index=None)][0]
    triplet_index = fs[TensorAttr(group_name=tuple(et), attr_name='triplet_index', index=None)][0]

    # Perform similarity computation
    modal_score = _compute_sim_scores(
        features_a=cp.asarray(edge_emb, dtype=cp.float32),
        features_b=cp.asarray(query_emb, dtype=cp.float32).reshape(1, -1),
        metric="cosine"
    )

    # Use CuPy to update scores in a flat buffer first
    score_map['|'.join(et)] = cudf.DataFrame({
        'triplet_index': triplet_index,
        'score': modal_score
    })

    print(modal_score.max(), modal_score.min(), modal_score.mean())

# Combine all modal score dataframes in one go (faster than sequential merging)
if score_map:
    all_modal_scores = cudf.concat(score_map.values())
    sim_score_df = sim_score_df.merge(all_modal_scores, on='triplet_index', how='left', suffixes=('', '_new'))
    sim_score_df['score'] = sim_score_df['score_new'].fillna(sim_score_df['score'])
    sim_score_df = sim_score_df.drop(columns=['score_new'])
    sim_score_df = sim_score_df.sort_values('triplet_index').reset_index(drop=True)

sim_score_df

Processing edge type: ['drug', 'carrier', 'gene/protein']
0.610337 0.56907153 0.59043986
Processing edge type: ['drug', 'enzyme', 'gene/protein']
0.61401385 0.55352855 0.5838153
Processing edge type: ['drug', 'target', 'gene/protein']
0.6776841 0.54225314 0.60850626
Processing edge type: ['drug', 'transporter', 'gene/protein']
0.6388326 0.54529464 0.5889088
Processing edge type: ['gene/protein', 'associated with', 'disease']
0.73586786 0.62134904 0.67582864
Processing edge type: ['gene/protein', 'interacts with', 'molecular_function']
0.65209794 0.5007789 0.5634771
Processing edge type: ['gene/protein', 'interacts with', 'cellular_component']
0.6345832 0.4931153 0.5590816
Processing edge type: ['gene/protein', 'interacts with', 'biological_process']
0.6741964 0.5110974 0.58790225
Processing edge type: ['gene/protein', 'carrier', 'drug']
0.60508686 0.57738614 0.5940925
Processing edge type: ['gene/protein', 'enzyme', 'drug']
0.61281925 0.56178504 0.5892764
Processing edge type: ['gene/p

,triplet_index,score
0,0,0.579141
1,1,0.603209
2,2,0.569072
3,3,0.610337
4,4,0.614014
...,...,...
11267,11267,0.548734
11268,11268,0.587184
11269,11269,0.557806
11270,11270,0.596282


In [51]:
e_prizes = sim_score_df.score.to_cupy()
cp.unique(e_prizes, return_inverse=True)

(array([0.48611093, 0.4931153 , 0.49485004, ..., 0.73586786, 0.73871225,
        0.7421171 ], dtype=float32),
 array([5360, 8368, 3891, ..., 2360, 7644, 9162]))

In [52]:
topk_e = 5
c_const = 0.1

# ==============

unique_prizes, inverse_indices = cp.unique(e_prizes, return_inverse=True)
topk_e = min(topk_e, len(triplet_index_sorted))
topk_e_values = unique_prizes[cp.argsort(-unique_prizes)[:topk_e]]
e_prizes[e_prizes < topk_e_values[-1]] = 0.0
last_topk_e_value = topk_e
for k in range(topk_e):
    indices = inverse_indices == (unique_prizes == topk_e_values[k]).nonzero()[0]
    value = min((topk_e - k) / indices.sum().item(), last_topk_e_value)
    e_prizes[indices] = value
    last_topk_e_value = value * (1 - c_const)

e_prizes

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [55]:
e_prizes

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [56]:
cp.unique(e_prizes)

array([0., 1., 2., 3., 4., 5.], dtype=float32)

In [47]:
sim_score_df.score.max(), sim_score_df.score.min(), sim_score_df.score.mean(), sim_score_df.score.median()

(0.7421171, 0.48611093, 0.5858773330769868, 0.5809487700462341)

In [43]:
score_map.keys()

dict_keys(['gene/protein'])

In [40]:
score_map

{'gene/protein':       triplet_index  sim_score
 0              8182   0.582982
 1              8183   0.572382
 2              8184   0.586945
 3              8185   0.587526
 4              8186   0.601592
 ...             ...        ...
 3085          11267   0.548734
 3086          11268   0.587184
 3087          11269   0.557806
 3088          11270   0.596282
 3089          11271   0.611896
 
 [3090 rows x 2 columns]}

In [ ]:
edge_emb.shape

In [ ]:
triplet_index.shape

In [ ]:
modal_score